In [1]:
import math
from collections import OrderedDict
from functools import partial
from typing import Any, Callable, Dict, List, NamedTuple, Optional, Tuple, Union
from types import FunctionType
import torch
import torch.nn as nn
import pytorch_lightning as pl

# Vision Transformer

In [2]:
class ConvStemConfig(NamedTuple):
    out_channels: int = 64
    kernel_size: int = 3
    stride: int = 2
    norm_layer: Callable[..., nn.Module] = nn.BatchNorm2d
    activation_layer: Callable[..., nn.Module] = nn.ReLU

In [3]:
def log_api_usage_once(obj: Any) -> None:

    """
    Logs API usage(module and name) within an organization.
    In a large ecosystem, it's often useful to track the PyTorch and
    TorchVision APIs usage. This API provides the similar functionality to the
    logging module in the Python stdlib. It can be used for debugging purpose
    to log which methods are used and by default it is inactive, unless the user
    manually subscribes a logger via the `SetAPIUsageLogger method <https://github.com/pytorch/pytorch/blob/eb3b9fe719b21fae13c7a7cf3253f970290a573e/c10/util/Logging.cpp#L114>`_.
    Please note it is triggered only once for the same API call within a process.
    It does not collect any data from open-source users since it is no-op by default.
    For more information, please refer to
    * PyTorch note: https://pytorch.org/docs/stable/notes/large_scale_deployments.html#api-usage-logging;
    * Logging policy: https://github.com/pytorch/vision/issues/5052;

    Args:
        obj (class instance or method): an object to extract info from.
    """
    module = obj.__module__
    if not module.startswith("torchvision"):
        module = f"torchvision.internal.{module}"
    name = obj.__class__.__name__
    if isinstance(obj, FunctionType):
        name = obj.__name__
    torch._C._log_api_usage_once(f"{module}.{name}")

In [4]:
def make_ntuple(x: Any, n: int) -> Tuple[Any, ...]:
    """
    Make n-tuple from input x. If x is an iterable, then we just convert it to tuple.
    Otherwise, we will make a tuple of length n, all with value of x.
    reference: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/utils.py#L8

    Args:
        x (Any): input value
        n (int): length of the resulting tuple
    """
    if isinstance(x, collections.abc.Iterable):
        return tuple(x)
    return tuple(repeat(x, n))

In [5]:
def expand_index_like(index: torch.Tensor, tokens: torch.Tensor) -> torch.Tensor:
    """Expands the index along the last dimension of the input tokens.

    Args:
        index:
            Index tensor with shape (batch_size, idx_length) where each entry is
            an index in [0, sequence_length).
        tokens:
            Tokens tensor with shape (batch_size, sequence_length, dim).

    Returns:
        Index tensor with shape (batch_size, idx_length, dim) where the original
        indices are repeated dim times along the last dimension.

    """
    dim = tokens.shape[-1]
    index = index.unsqueeze(-1).expand(-1, -1, dim)
    return index

In [6]:
def get_at_index(tokens: torch.Tensor, index: torch.Tensor) -> torch.Tensor:
    """Selects tokens at index.

    Args:
        tokens:
            Token tensor with shape (batch_size, sequence_length, dim).
        index:
            Index tensor with shape (batch_size, index_length) where each entry is
            an index in [0, sequence_length).

    Returns:
        Token tensor with shape (batch_size, index_length, dim) containing the
        selected tokens.

    """
    index = expand_index_like(index, tokens)
    return torch.gather(tokens, 1, index)

In [7]:
class ConvNormActivation(torch.nn.Sequential):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[int, Tuple[int, ...]] = 3,
        stride: Union[int, Tuple[int, ...]] = 1,
        padding: Optional[Union[int, Tuple[int, ...], str]] = None,
        groups: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.BatchNorm2d,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        dilation: Union[int, Tuple[int, ...]] = 1,
        inplace: Optional[bool] = True,
        bias: Optional[bool] = None,
        conv_layer: Callable[..., torch.nn.Module] = torch.nn.Conv2d,
    ) -> None:

        if padding is None:
            if isinstance(kernel_size, int) and isinstance(dilation, int):
                padding = (kernel_size - 1) // 2 * dilation
            else:
                _conv_dim = len(kernel_size) if isinstance(kernel_size, Sequence) else len(dilation)
                kernel_size = make_ntuple(kernel_size, _conv_dim)
                dilation = make_ntuple(dilation, _conv_dim)
                padding = tuple((kernel_size[i] - 1) // 2 * dilation[i] for i in range(_conv_dim))
        if bias is None:
            bias = norm_layer is None

        layers = [
            conv_layer(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                dilation=dilation,
                groups=groups,
                bias=bias,
            )
        ]

        if norm_layer is not None:
            layers.append(norm_layer(out_channels))

        if activation_layer is not None:
            params = {} if inplace is None else {"inplace": inplace}
            layers.append(activation_layer(**params))
        super().__init__(*layers)
        log_api_usage_once(self)
        self.out_channels = out_channels

        if self.__class__ == ConvNormActivation:
            warnings.warn(
                "Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead."
            )

In [8]:
class Conv2dNormActivation(ConvNormActivation):
    """
    Configurable block used for Convolution2d-Normalization-Activation blocks.

    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the Convolution-Normalization-Activation block
        kernel_size: (int, optional): Size of the convolving kernel. Default: 3
        stride (int, optional): Stride of the convolution. Default: 1
        padding (int, tuple or str, optional): Padding added to all four sides of the input. Default: None, in which case it will be calculated as ``padding = (kernel_size - 1) // 2 * dilation``
        groups (int, optional): Number of blocked connections from input channels to output channels. Default: 1
        norm_layer (Callable[..., torch.nn.Module], optional): Norm layer that will be stacked on top of the convolution layer. If ``None`` this layer won't be used. Default: ``torch.nn.BatchNorm2d``
        activation_layer (Callable[..., torch.nn.Module], optional): Activation function which will be stacked on top of the normalization layer (if not None), otherwise on top of the conv layer. If ``None`` this layer won't be used. Default: ``torch.nn.ReLU``
        dilation (int): Spacing between kernel elements. Default: 1
        inplace (bool): Parameter for the activation layer, which can optionally do the operation in-place. Default ``True``
        bias (bool, optional): Whether to use bias in the convolution layer. By default, biases are included if ``norm_layer is None``.

    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[int, Tuple[int, int]] = 3,
        stride: Union[int, Tuple[int, int]] = 1,
        padding: Optional[Union[int, Tuple[int, int], str]] = None,
        groups: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.BatchNorm2d,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        dilation: Union[int, Tuple[int, int]] = 1,
        inplace: Optional[bool] = True,
        bias: Optional[bool] = None,
    ) -> None:

        super().__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups,
            norm_layer,
            activation_layer,
            dilation,
            inplace,
            bias,
            torch.nn.Conv2d,
        )

In [9]:
class MLP(torch.nn.Sequential):
    """This block implements the multi-layer perceptron (MLP) module.

    Args:
        in_channels (int): Number of channels of the input
        hidden_channels (List[int]): List of the hidden channel dimensions
        norm_layer (Callable[..., torch.nn.Module], optional): Norm layer that will be stacked on top of the linear layer. If ``None`` this layer won't be used. Default: ``None``
        activation_layer (Callable[..., torch.nn.Module], optional): Activation function which will be stacked on top of the normalization layer (if not None), otherwise on top of the linear layer. If ``None`` this layer won't be used. Default: ``torch.nn.ReLU``
        inplace (bool, optional): Parameter for the activation layer, which can optionally do the operation in-place.
            Default is ``None``, which uses the respective default values of the ``activation_layer`` and Dropout layer.
        bias (bool): Whether to use bias in the linear layer. Default ``True``
        dropout (float): The probability for the dropout layer. Default: 0.0
    """

    def __init__(
        self,
        in_channels: int,
        hidden_channels: List[int],
        norm_layer: Optional[Callable[..., torch.nn.Module]] = None,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        inplace: Optional[bool] = None,
        bias: bool = True,
        dropout: float = 0.0,
    ):
        # The addition of `norm_layer` is inspired from the implementation of TorchMultimodal:
        # https://github.com/facebookresearch/multimodal/blob/5dec8a/torchmultimodal/modules/layers/mlp.py
        params = {} if inplace is None else {"inplace": inplace}

        layers = []
        in_dim = in_channels
        for hidden_dim in hidden_channels[:-1]:
            layers.append(torch.nn.Linear(in_dim, hidden_dim, bias=bias))
            if norm_layer is not None:
                layers.append(norm_layer(hidden_dim))
            layers.append(activation_layer(**params))
            layers.append(torch.nn.Dropout(dropout, **params))
            in_dim = hidden_dim

        layers.append(torch.nn.Linear(in_dim, hidden_channels[-1], bias=bias))
        layers.append(torch.nn.Dropout(dropout, **params))

        super().__init__(*layers)
        log_api_usage_once(self)

In [10]:
class MLPBlock(MLP):
    """Transformer MLP block."""

    _version = 2

    def __init__(self, in_dim: int, mlp_dim: int, dropout: float):
        super().__init__(in_dim, [mlp_dim, in_dim], activation_layer=nn.GELU, inplace=None, dropout=dropout)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.normal_(m.bias, std=1e-6)

    def _load_from_state_dict(
        self,
        state_dict,
        prefix,
        local_metadata,
        strict,
        missing_keys,
        unexpected_keys,
        error_msgs,
    ):
        version = local_metadata.get("version", None)

        if version is None or version < 2:
            # Replacing legacy MLPBlock with MLP. See https://github.com/pytorch/vision/pull/6053
            for i in range(2):
                for type in ["weight", "bias"]:
                    old_key = f"{prefix}linear_{i+1}.{type}"
                    new_key = f"{prefix}{3*i}.{type}"
                    if old_key in state_dict:
                        state_dict[new_key] = state_dict.pop(old_key)

        super()._load_from_state_dict(
            state_dict,
            prefix,
            local_metadata,
            strict,
            missing_keys,
            unexpected_keys,
            error_msgs,
        )

In [11]:
class EncoderBlock(nn.Module):
    """Transformer encoder block."""

    def __init__(
        self,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        self.num_heads = num_heads

        # Attention block
        self.ln_1 = norm_layer(hidden_dim)
        self.self_attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=attention_dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

        # MLP block
        self.ln_2 = norm_layer(hidden_dim)
        self.mlp = MLPBlock(hidden_dim, mlp_dim, dropout)

    def forward(self, input: torch.Tensor):
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        x = self.ln_1(input)
        x, _ = self.self_attention(x, x, x, need_weights=False)
        x = self.dropout(x)
        x = x + input

        y = self.ln_2(x)
        y = self.mlp(y)
        return x + y

In [12]:
class Encoder(nn.Module):
    """Transformer Model Encoder for sequence to sequence translation."""

    def __init__(
        self,
        seq_length: int,
        num_layers: int,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        num_cls_tokens: int,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6)
    ):
        super().__init__()
        self.num_cls_tokens = num_cls_tokens
        self.seq_length = seq_length
        # Note that batch_size is on the first dim because
        # we have batch_first=True in nn.MultiAttention() by default

        self.pos_embedding = nn.Parameter(torch.empty(1, seq_length, hidden_dim).normal_(std=0.02))  # from BERT

        self.dropout = nn.Dropout(dropout)
        layers: OrderedDict[str, nn.Module] = OrderedDict()
        for i in range(num_layers):
            layers[f"encoder_layer_{i}"] = EncoderBlock(
                num_heads,
                hidden_dim,
                mlp_dim,
                dropout,
                attention_dropout,
                norm_layer,
            )
        self.layers = nn.Sequential(layers)
        self.ln = norm_layer(hidden_dim)

    def forward(self,
                input: torch.Tensor,
                idx_keep: Optional[torch.Tensor] = None
               ):

        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")

        input = input + self._interpolate_pos_encoding(input,self.num_cls_tokens)
        if idx_keep is not None:
            input = get_at_index(input, idx_keep)
        return self.ln(self.layers(self.dropout(input)))

    def _interpolate_pos_encoding(self,
                                  input: torch.Tensor,
                                  num_cls_tokens: int = 3):
        """Returns the interpolated positional embedding for the given input.

        This function interpolates self.pos_embedding for all tokens in the input,
        ignoring the class token. This allows encoding variable sized images.

        Args:
            input:
               Input tensor with shape (batch_size, num_sequences).
            num_cls_tokens:
               number of classification tokens prepended to the
        """

        npatch = input.shape[1] - num_cls_tokens
        N = self.pos_embedding.shape[1] - num_cls_tokens
        diff = num_cls_tokens -1
        if npatch == N:
            return self.pos_embedding

        else:
            npatch += diff
            class_emb = self.pos_embedding[:, 1]
            pos_embedding = self.pos_embedding[:, 1+diff:]
        dim = input.shape[-1]
        pos_embedding = nn.functional.interpolate(
            pos_embedding.reshape(1, int(math.sqrt(N)), int(math.sqrt(N)), dim).permute(0, 3, 1, 2),
            scale_factor=math.sqrt(npatch / N),
            mode="bicubic",
        )
        pos_embedding = pos_embedding.permute(0, 2, 3, 1).view(1, -1, dim)
        return torch.cat((class_emb.unsqueeze(0), pos_embedding), dim=1)

In [13]:
class VisionTransformer(nn.Module):
    """Vision Transformer as per https://arxiv.org/abs/2010.11929."""
    # defualt settings vits16
    def __init__(
        self,
        image_size: int = 224,
        patch_size: int = 16,
        num_layers: int = 12,
        num_heads: int = 3,
        hidden_dim: int = 192,
        mlp_dim: int = 768,
        dropout: float = 0.0,
        attention_dropout: float = 0.0,
        num_classes: int = 1000,
        num_cls_tokens: int = 3,
        representation_size: Optional[int] = None,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
        conv_stem_configs: Optional[List[ConvStemConfig]] = None,
    ):
        super().__init__()
        log_api_usage_once(self)
        torch._assert(image_size % patch_size == 0, "Input shape indivisible by patch size!")
        self.image_size = image_size
        self.patch_size = patch_size
        self.hidden_dim = hidden_dim
        self.mlp_dim = mlp_dim
        self.attention_dropout = attention_dropout
        self.dropout = dropout
        self.num_classes = num_classes
        self.representation_size = representation_size
        self.norm_layer = norm_layer
        self.num_cls_tokens = num_cls_tokens

        if conv_stem_configs is not None:
            # As per https://arxiv.org/abs/2106.14881
            seq_proj = nn.Sequential()
            prev_channels = 3
            for i, conv_stem_layer_config in enumerate(conv_stem_configs):
                seq_proj.add_module(
                    f"conv_bn_relu_{i}",
                    Conv2dNormActivation(
                        in_channels=prev_channels,
                        out_channels=conv_stem_layer_config.out_channels,
                        kernel_size=conv_stem_layer_config.kernel_size,
                        stride=conv_stem_layer_config.stride,
                        norm_layer=conv_stem_layer_config.norm_layer,
                        activation_layer=conv_stem_layer_config.activation_layer,
                    ),
                )
                prev_channels = conv_stem_layer_config.out_channels
            seq_proj.add_module(
                "conv_last", nn.Conv2d(in_channels=prev_channels, out_channels=hidden_dim, kernel_size=1)
            )
            self.conv_proj: nn.Module = seq_proj
        else:
            self.conv_proj = nn.Conv2d(
                in_channels=3, out_channels=hidden_dim, kernel_size=patch_size, stride=patch_size
            )

        self.seq_length = (image_size // patch_size) ** 2
        # Add a class token
        self.class_token = nn.Parameter(torch.zeros(1, self.num_cls_tokens, hidden_dim))
        self.seq_length += self.num_cls_tokens

        self.encoder = Encoder(
            self.seq_length,
            num_layers,
            num_heads,
            hidden_dim,
            mlp_dim,
            dropout,
            attention_dropout,
            num_cls_tokens,
            norm_layer)
        #self.seq_length = seq_length #+ self.num_cls_tokens

        heads_layers: OrderedDict[str, nn.Module] = OrderedDict()
        if representation_size is None:
            heads_layers["head"] = nn.Linear(hidden_dim, num_classes)
        else:
            heads_layers["pre_logits"] = nn.Linear(hidden_dim, representation_size)
            heads_layers["act"] = nn.Tanh()
            heads_layers["head"] = nn.Linear(representation_size, num_classes)

        self.heads = nn.Sequential(heads_layers)

        if isinstance(self.conv_proj, nn.Conv2d):
            # Init the patchify stem
            fan_in = self.conv_proj.in_channels * self.conv_proj.kernel_size[0] * self.conv_proj.kernel_size[1]
            nn.init.trunc_normal_(self.conv_proj.weight, std=math.sqrt(1 / fan_in))
            if self.conv_proj.bias is not None:
                nn.init.zeros_(self.conv_proj.bias)
        elif self.conv_proj.conv_last is not None and isinstance(self.conv_proj.conv_last, nn.Conv2d):
            # Init the last 1x1 conv of the conv stem
            nn.init.normal_(
                self.conv_proj.conv_last.weight, mean=0.0, std=math.sqrt(2.0 / self.conv_proj.conv_last.out_channels)
            )
            if self.conv_proj.conv_last.bias is not None:
                nn.init.zeros_(self.conv_proj.conv_last.bias)

        if hasattr(self.heads, "pre_logits") and isinstance(self.heads.pre_logits, nn.Linear):
            fan_in = self.heads.pre_logits.in_features
            nn.init.trunc_normal_(self.heads.pre_logits.weight, std=math.sqrt(1 / fan_in))
            nn.init.zeros_(self.heads.pre_logits.bias)

        if isinstance(self.heads.head, nn.Linear):
            nn.init.zeros_(self.heads.head.weight)
            nn.init.zeros_(self.heads.head.bias)

    def _process_input(self, x: torch.Tensor) -> torch.Tensor:
        n, c, h, w = x.shape
        p = self.patch_size
        #torch._assert(h == self.image_size, f"Wrong image height! Expected {self.image_size} but got {h}!")
        #torch._assert(w == self.image_size, f"Wrong image width! Expected {self.image_size} but got {w}!")
        n_h = h // p
        n_w = w // p

        # (n, c, h, w) -> (n, hidden_dim, n_h, n_w)
        x = self.conv_proj(x)
        # (n, hidden_dim, n_h, n_w) -> (n, hidden_dim, (n_h * n_w))
        x = x.reshape(n, self.hidden_dim, n_h * n_w)

        # (n, hidden_dim, (n_h * n_w)) -> (n, (n_h * n_w), hidden_dim)
        # The self attention layer expects inputs in the format (N, S, E)
        # where S is the source sequence length, N is the batch size, E is the
        # embedding dimension
        x = x.permute(0, 2, 1)

        return x

    def forward(self, x: torch.Tensor,
                branch: str = 'target',
                idx_keep: Optional[torch.Tensor] = None):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        # Expand the class token to the full batch
        if branch == 'target':
            batch_class_token = self.class_token.expand(n, -1, -1)
            x = torch.cat([batch_class_token, x], dim=1)
            x = self.encoder(x)
            return x[:,0:self.num_cls_tokens]
        elif branch == 'anchor':
            batch_class_token = self.class_token.expand(n, -1, -1)
            x = torch.cat([batch_class_token, x], dim=1)
            x = torch.cat((x[:,:1],x[:,self.num_cls_tokens:]),dim = 1)
            x = self.encoder(x,idx_keep=idx_keep)
            return x[:,0]

        # Classifier "token" as used by standard language architectures
        # else:
        #x = x[:, 0]
        #x = self.heads(x)

        # return x

In [14]:
# stemconfig = [ConvStemConfig(out_channels = 64, kernel_size = 3 , stride = 2) for i in range(4)]
transformer = VisionTransformer(image_size=224,
                                patch_size=16,
                                num_layers=12,
                                num_heads=3,
                                hidden_dim=192,
                                mlp_dim=768,
                                dropout=0.0,
                                attention_dropout=0.0,
                                num_cls_tokens=3,
#                                 conv_stem_configs=stemconfig
                               )

In [15]:
x= torch.randn(64,3,224,224)
transformer(x,branch='anchor')

tensor([[ 0.5596, -1.3186, -2.4964,  ...,  1.8443,  1.0027,  0.7661],
        [-1.4058, -0.5556, -1.9416,  ...,  0.8630,  0.3313, -0.3766],
        [ 0.2584, -1.4279, -0.2105,  ...,  2.4241,  0.0840, -1.0784],
        ...,
        [ 1.2345, -1.6663, -0.9487,  ...,  1.0602, -0.1681, -0.2912],
        [-0.0206,  0.6050, -1.1963,  ...,  1.8352,  1.9001, -0.1919],
        [-0.2642, -1.6112, -1.3572,  ...,  1.9571, -0.2857, -0.1676]],
       grad_fn=<SelectBackward0>)

# ChexMSN

In [16]:
import copy

In [17]:
def count_parameters(model:nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
def deactivate_requires_grad(model: nn.Module):
    """Deactivates the requires_grad flag for all parameters of a model.

    This has the same effect as permanently executing the model within a `torch.no_grad()`
    context. Use this method to disable gradient computation and therefore
    training for a model.

    Examples:
        >>> backbone = resnet18()
        >>> deactivate_requires_grad(backbone)
    """
    for param in model.parameters():
        param.requires_grad = False

In [19]:
@torch.no_grad()
def update_momentum(model: nn.Module, model_ema: nn.Module, m: float):
    """Updates parameters of `model_ema` with Exponential Moving Average of `model`

    Momentum encoders are a crucial component fo models such as MoCo or BYOL.

    Examples:
        >>> backbone = resnet18()
        >>> projection_head = MoCoProjectionHead()
        >>> backbone_momentum = copy.deepcopy(moco)
        >>> projection_head_momentum = copy.deepcopy(projection_head)
        >>>
        >>> # update momentum
        >>> update_momentum(moco, moco_momentum, m=0.999)
        >>> update_momentum(projection_head, projection_head_momentum, m=0.999)
    """
    for model_ema, model in zip(model_ema.parameters(), model.parameters()):
        model_ema.data = model_ema.data * m + model.data * (1.0 - m)

In [20]:
def random_token_mask(
    size: Tuple[int, int],
    mask_ratio: float = 0.15,
    mask_class_token: bool = False,
    device: Optional[Union[torch.device, str]] = None,
) -> torch.Tensor:
    """Creates random token masks.

    Args:
        size:
            Size of the token batch for which to generate masks.
            Should be (batch_size, sequence_length).
        mask_ratio:
            Percentage of tokens to mask.
        mask_class_token:
            If False the class token is never masked. If True the class token
            might be masked.
        device:
            Device on which to create the index masks.

    Returns:
        A (index_keep, index_mask) tuple where each index is a tensor.
        index_keep contains the indices of the unmasked tokens and has shape
        (batch_size, num_keep). index_mask contains the indices of the masked
        tokens and has shape (batch_size, sequence_length - num_keep).
        num_keep is equal to sequence_length * (1- mask_ratio).

    """
    batch_size, sequence_length = size
    num_keep = int(sequence_length * (1 - mask_ratio))

    noise = torch.rand(batch_size, sequence_length, device=device)
    if not mask_class_token and sequence_length > 0:
        # make sure that class token is not masked
        noise[:, 0] = -1
        num_keep = max(1, num_keep)

    # get indices of tokens to keep
    indices = torch.argsort(noise, dim=1)
    idx_keep = indices[:, :num_keep]
    idx_mask = indices[:, num_keep:]

    return idx_keep, idx_mask

In [21]:
class DenseBlock(nn.Module):
    def __init__(self,
                 in_features: int = 768,
                 out_features: int = 2048,
               bias: bool = False,
              ) -> None:

        super().__init__()

        self.dense_block = nn.Sequential(nn.Linear(in_features= in_features,
                                                   out_features= out_features,
                                               bias=bias),
                                         nn.LayerNorm(normalized_shape= out_features),
                                         nn.GELU()
                                        )
    def forward(self,
                x: torch.Tensor
                ) -> torch.Tensor:
        x = self.dense_block(x)
        return x

In [22]:
class ProjectionHead(nn.Module):
    def __init__(self,
                 in_features: int = 192,
                 hidden_features: int = 768,
                 out_features: int = 192,
                 bias : bool = False
                 ) -> None:

        super().__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.projection_head = nn.Sequential(DenseBlock(in_features= self.in_features,
                                                    out_features= self.hidden_features,
                                                    bias= bias
                                                    ),
                                         DenseBlock(in_features= self.hidden_features,
                                                    out_features= self.hidden_features,
                                                    bias= bias
                                                    ),
                                         nn.Linear(in_features= self.hidden_features,
                                                   out_features= self.out_features,
                                                   bias= bias
                                                   )
                                         )

    def forward(self,
                x: torch.Tensor
                ) -> torch.Tensor:
        x = self.projection_head(x)
        return x

In [23]:
projection_head = ProjectionHead(in_features=192,
                                 hidden_features=768,
                                 out_features=192)

In [24]:
class ChexMSN(nn.Module):
    def __init__(self,
               backbone: nn.Module,
               projection_head: nn.Module,
               masking_ratio: float = 0.15,
               ema_p: float = 0.996,
               focal: bool = True
              ) -> None:
        super().__init__()

        self.masking_ratio = masking_ratio
        self.ema_p = ema_p
        self.focal = focal

        self.backbone = backbone
        self.projection_head = projection_head

        self.target_backbone = copy.deepcopy(self.backbone)
        self.target_projection_head = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.target_backbone)
        deactivate_requires_grad(self.target_projection_head)



    def forward(self,
                views:list[torch.Tensor],
                focal: bool = True
                ) -> tuple[torch.Tensor]:

        update_momentum(model= self.backbone,
                        model_ema= self.target_backbone,
                        m = self.ema_p,
                       )
        update_momentum(model= self.projection_head,
                        model_ema= self.target_projection_head,
                        m = self.ema_p,
                       )
        projections = self._forward_all(batch=views,focal=self.focal)
        

        return projections

    def _target_forward(self,
                        view: torch.Tensor
                        ) -> torch.Tensor:

        target_encodings = self.target_backbone(x= view,
                                                branch='target')
        target_projections = self.target_projection_head(x= target_encodings)

        return target_projections


    def _anchor_forward(self,
                        view: torch.Tensor
                        ) -> torch.Tensor:

        batch_size, _, _, width = view.shape
        seq_length = (width // self.backbone.patch_size) ** 2
        idx_keep, idx_mask = random_token_mask(size= (view.shape[0],seq_length),
                                               mask_ratio= self.masking_ratio,
                                               device=torch.device("cuda"))

        anchor_encodings = self.backbone(x= view,
                                         branch= 'anchor',
                                         idx_keep= idx_keep)
        anchor_projections = self.projection_head(x= anchor_encodings)

        return anchor_projections
    
    
    def _forward_all(self,
                     batch: list,
                     focal: bool = True
                     ) -> torch.Tensor:
        
        target_view = batch[0][0].to('cuda')
        anchor_view_sim = batch[0][1].to('cuda')
        focal_views_sim = torch.concat(batch[0][2:],dim=0).to('cuda')
        anchor_view_age = batch[1][1].to('cuda')
        focal_views_age = torch.concat(batch[1][2:],dim=0).to('cuda')
        anchor_view_gender = batch[2][1].to('cuda')
        focal_views_gender = torch.concat(batch[2][2:],dim=0).to('cuda')
        
        
        target_projections = self._target_forward(target_view)
        
        anchor_projections_sim = self._anchor_forward(anchor_view_sim)
        if focal:
            anchor_focal_projections_sim = self._anchor_forward(focal_views_sim)
            similarity_projections = self._arrange_tokens(anchor_projections_sim,
                                                          anchor_focal_projections_sim,
                                                          num_focal = 10)
                                                          
        
        anchor_projections_age = self._anchor_forward(anchor_view_age)
        if focal:
            anchor_focal_projections_age =  self._anchor_forward(focal_views_age)
            age_projections = self._arrange_tokens(anchor_projections_age,
                                                   anchor_focal_projections_age,
                                                   num_focal = 10)
                                                    
        
        
        anchor_projections_gender = self._anchor_forward(anchor_view_gender)
        if focal:
            anchor_focal_projections_gender = self._anchor_forward(focal_views_gender)  
            gender_projections = self._arrange_tokens(anchor_projections_gender,
                                                      anchor_focal_projections_gender,
                                                      num_focal = 10)
                                                          
        if focal:
            anchor_projections = torch.stack((similarity_projections,
                                              age_projections,
                                              gender_projections
                                              ),
                                         dim= 0)
        else:
            anchor_projections = torch.stack((anchor_projections_sim,
                                              anchor_projections_age,
                                              anchor_projections_gender
                                              ),
                                             dim= 1)       

        return (anchor_projections,
                target_projections)
    
    def _arrange_tokens(self,
                        tensor1: torch.Tensor,
                        tensor2:torch.Tensor,
                        num_focal: int = 10
                        ) ->torch.Tensor:

        a = torch.stack(torch.split(tensor1,1),0)
        b = torch.stack(torch.split(tensor2,num_focal),0)
        c = torch.cat((a.expand(-1,num_focal,-1),b),dim=1)[:,num_focal-1:]
        arranged_tokens = torch.cat(c.split(1),1).squeeze(0)
        return arranged_tokens

In [25]:
msn = ChexMSN(backbone=transformer,
             projection_head=projection_head,
             masking_ratio=0.15,
             ema_p=0.996,
             focal=True)

# MSN Loss

In [26]:
import math
import warnings
from typing import Union

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [27]:
def prototype_probabilities(queries: torch.Tensor,
                            prototypes: torch.Tensor,
                            temperature: float,
                            ) -> torch.Tensor:
    """Returns probability for each query to belong to each prototype.

    Args:
        queries:
            Tensor with shape (batch_size, dim), projection head output
        prototypes:
            Tensor with shape (num_prototypes, dim)
        temperature:
            Inverse scaling factor for the similarity.

    Returns:
        Probability tensor with shape (batch_size, num_prototypes) which sums to 1 along
        the num_prototypes dimension.

    """                           
    return F.softmax(torch.matmul(queries, prototypes.T) / temperature, dim=1)

In [28]:
def sharpen(probabilities: torch.Tensor, 
            temperature: float
           ) -> torch.Tensor:
    """Sharpens the probabilities with the given temperature.

    Args:
        probabilities:
            Tensor with shape (batch_size, dim)
        temperature:
            Temperature in (0, 1]. Lower temperature results in stronger sharpening (
            output probabilities are less uniform).
    Returns:
        Probabilities tensor with shape (batch_size, num_prototypes).

    """
    probabilities = probabilities ** (1.0 / temperature)
    probabilities /= torch.sum(probabilities, dim=1, keepdim=True)
    return probabilities

In [29]:
@torch.no_grad()
def sinkhorn(probabilities: torch.Tensor,
             iterations: int = 3,
             gather_distributed: bool = False,
            ) -> torch.Tensor:
    """Runs sinkhorn normalization on the probabilities as described in [0].

    Code inspired by [1].

    - [0]: Masked Siamese Networks, 2022, https://arxiv.org/abs/2204.07141
    - [1]: https://github.com/facebookresearch/msn

    Args:
        probabilities:
            Probabilities tensor with shape (batch_size, num_prototypes).
        iterations:
            Number of iterations of the sinkhorn algorithms. Set to 0 to disable.
        gather_distributed:
            If True then features from all gpus are gathered during normalization.
    Returns:
        A normalized probabilities tensor.

    """
    if iterations <= 0:
        return probabilities


    num_targets, num_prototypes = probabilities.shape
    probabilities = probabilities.T
    sum_probabilities = torch.sum(probabilities)

    probabilities = probabilities / sum_probabilities

    for _ in range(iterations):
        # normalize rows
        row_sum = torch.sum(probabilities, dim=1, keepdim=True)

        probabilities /= row_sum
        probabilities /= num_prototypes

        # normalize columns
        probabilities /= torch.sum(probabilities, dim=0, keepdim=True)
        probabilities /= num_targets

    probabilities *= num_targets
    return probabilities.T

In [30]:
def regularization_loss(mean_anchor_probs: torch.Tensor
                       ) -> torch.Tensor:
    """Calculates mean entropy regularization loss."""
    loss = -torch.sum(torch.log(mean_anchor_probs ** (-mean_anchor_probs)))
    loss += math.log(float(len(mean_anchor_probs)))
    return loss

In [31]:
class MSNLoss(nn.Module):
    def __init__(self,
                 temperature: float = 0.1,
                 sinkhorn_iterations: int = 3,
                 similarity_weight: float = 1.0,
                 age_weight: float = 1.0,
                 gender_weight: float = 1.0,
                 regularization_weight: float = 1.0,
                 gather_distributed: bool = False,
               ) -> None:

        super().__init__()
        
        self.temperature = temperature
        self.sinkhorn_iterations = sinkhorn_iterations
        self.similarity_weight = similarity_weight
        self.age_weight = age_weight
        self.gender_weight = gender_weight
        self.regularization_weight = regularization_weight

        
    def forward(self,
                anchors: torch.Tensor,
                targets: torch.Tensor,
                prototypes: torch.Tensor,
                target_sharpen_temperature: float = 0.25,
                focal: bool = True
                      ) -> torch.Tensor:
        
        similarity_loss = self.similarity_weight * self._forward_loss(anchors=anchors[0] if focal else anchors[:,0],
                                                                      targets=targets[:,0],
                                                                      prototypes=prototypes[0].weight.data
                                                                     )

        age_loss = self.age_weight * self._forward_loss(anchors=anchors[1] if focal else anchors[:,1],
                                                           targets=targets[:,1],
                                                           prototypes=prototypes[1].weight.data
                                                          )

        gender_loss = self.gender_weight * self._forward_loss(anchors=anchors[2] if focal else anchors[:,2],
                                                              targets=targets[:,2],
                                                              prototypes=prototypes[2].weight.data
                                                             )

        loss = similarity_loss + age_loss + gender_loss
        return loss
    
    def _forward_loss(self,
                anchors: torch.Tensor,
                targets: torch.Tensor,
                prototypes: torch.Tensor,
                target_sharpen_temperature: float = 0.25,
               ) -> torch.Tensor:

        num_views = anchors.shape[0] // targets.shape[0]
        anchors = F.normalize(anchors, dim=1)
        targets = F.normalize(targets, dim=1)
        prototypes = F.normalize(prototypes, dim=1)

        anchor_probs = prototype_probabilities(anchors, 
                                               prototypes, 
                                               temperature=self.temperature
                                              )

        with torch.no_grad():
            target_probs = prototype_probabilities(targets, 
                                                   prototypes, 
                                                   temperature=self.temperature
                                                   )
            target_probs = sharpen(target_probs, temperature=target_sharpen_temperature)
            if self.sinkhorn_iterations > 0:
                target_probs = sinkhorn(probabilities=target_probs,
                                        iterations=self.sinkhorn_iterations,
                                        )
            target_probs = torch.repeat_interleave(target_probs, repeats=num_views, dim=0)
        
        loss = torch.mean(torch.sum(torch.log(anchor_probs ** (-target_probs)), dim=1))

        # regularization loss
        if self.regularization_weight > 0:
            mean_anchor_probs = torch.mean(anchor_probs, dim=0)
            reg_loss = regularization_loss(mean_anchor_probs=mean_anchor_probs)
            loss += self.regularization_weight * reg_loss
         
        return loss

In [32]:
criterterion = MSNLoss(temperature=0.1,
                       sinkhorn_iterations=0,
                       similarity_weight=1.0,
                       age_weight=1.0,
                       gender_weight=1.0,
                       regularization_weight=0.0)

# MSN transforms

In [33]:
from typing import Dict, List, Optional, Tuple, Union

import torchvision.transforms as T
from PIL.Image import Image
from torch import Tensor


from lightly.transforms.multi_view_transform import MultiViewTransform
IMAGENET_STAT = {"mean":torch.tensor([0.4884, 0.4550, 0.4171]),
                 "std":torch.tensor([0.2596, 0.2530, 0.2556])}

MIMIC_NORMALIZE ={"mean":torch.tensor([0.4723, 0.4723, 0.4723]), 
                  "std":torch.tensor([0.3023, 0.3023, 0.3023])}

In [34]:
class MSNTransform(MultiViewTransform):
    """Implements the transformations for MSN [0].

    Input to this transform:
        PIL Image or Tensor.

    Output of this transform:
        List of Tensor of length 2 * random_views + focal_views. (12 by default)

    Applies the following augmentations by default:
        - Random resized crop
        - Random horizontal flip
        - ImageNet normalization

    Generates a set of random and focal views for each input image. The generated output
    is (views, target, filenames) where views is list with the following entries:
    [random_views_0, random_views_1, ..., focal_views_0, focal_views_1, ...].

    - [0]: Masked Siamese Networks, 2022: https://arxiv.org/abs/2204.07141

    Attributes:
        random_size:
            Size of the random image views in pixels.
        focal_size:
            Size of the focal image views in pixels.
        random_views:
            Number of random views to generate.
        focal_views:
            Number of focal views to generate.
        random_crop_scale:
            Minimum and maximum size of the randomized crops for the relative to random_size.
        focal_crop_scale:
            Minimum and maximum size of the randomized crops relative to focal_size.
        hf_prob:
            Probability that horizontal flip is applied.
        vf_prob:
            Probability that vertical flip is applied.
        normalize:
            Dictionary with 'mean' and 'std' for torchvision.transforms.Normalize.
    """

    def __init__(
        self,
        random_size: int = 224,
        focal_size: int = 96,
        random_views: int = 2,
        focal_views: int = 10,
        affine_dgrees: int = 15,
        affine_scale: Tuple[float,float]= (.9, 1.1),
        affine_shear: int = 0,
        affine_translate: Tuple[float,float] = (0.1, 0.1),
        random_crop_scale: Tuple[float, float] = (0.3, 1.0),
        focal_crop_scale: Tuple[float, float] = (0.05, 0.3),
        hf_prob: float = 0.5,
        vf_prob: float = 0.0,
        normalize: Dict[str, List[float]] = MIMIC_NORMALIZE,
    ):
        random_view_transform = MSNViewTransform(
            affine_dgrees=affine_dgrees,
            affine_scale=affine_scale,
            affine_shear=affine_shear,
            affine_translate=affine_translate,
            crop_size=random_size,
            crop_scale=random_crop_scale,
            hf_prob=hf_prob,
            vf_prob=vf_prob,
            normalize=normalize,
        )
        focal_view_transform = MSNViewTransform(
            affine_dgrees=affine_dgrees,
            affine_scale=affine_scale,
            affine_shear=affine_shear,
            affine_translate=affine_translate,
            crop_size=focal_size,
            crop_scale=focal_crop_scale,
            hf_prob=hf_prob,
            vf_prob=vf_prob,
            normalize=normalize,
        )
        transforms = [random_view_transform] * random_views
        transforms += [focal_view_transform] * focal_views
        super().__init__(transforms=transforms)

In [35]:
class MSNViewTransform:
    def __init__(
        self,
        affine_dgrees: int = 15,
        affine_scale: Tuple[float,float]= (.9, 1.1),
        affine_shear: int = 0,
        affine_translate: Tuple[float,float] = (0.1, 0.1),
        crop_size: int = 224,
        crop_scale: Tuple[float, float] = (0.3, 1.0),
        hf_prob: float = 0.5,
        vf_prob: float = 0.0,
        normalize: Dict[str, List[float]] = MIMIC_NORMALIZE,
    ):

        transform = [
            T.RandomAffine(degrees=affine_dgrees, 
                           scale=affine_scale, 
                           shear=affine_shear, 
                           translate=affine_translate),
            T.RandomResizedCrop(size=crop_size, scale=crop_scale),
            T.RandomHorizontalFlip(p=hf_prob),
            T.RandomVerticalFlip(p=vf_prob),
            T.ToTensor(),
            T.Normalize(mean=normalize["mean"], std=normalize["std"]),
        ]

        self.transform = T.Compose(transform)

    def __call__(self, image: Union[torch.Tensor, Image]) -> torch.Tensor:
        """
        Applies the transforms to the input image.

        Args:
            image:
                The input image to apply the transforms to.

        Returns:
            The transformed image.

        """
        transformed = self.transform(image)
        return transformed

# MSN dataset

In [36]:
import os
import torch
import torch.nn as nn 
import pytorch_lightning as pl

import random
from collections import Counter
from typing import Tuple, Optional
from torch import Tensor
from PIL import Image
from torch.utils.data import Dataset

import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.transforms as T
from tqdm import tqdm
from torch import Tensor
from copy import deepcopy
from typing import List, Tuple, Dict
from torch.utils.data import DataLoader



In [37]:
class ChexMSNDataset(Dataset):
    def __init__(self, 
                 data_dir: str,
                 transforms: nn.Module,
                 same = True
                 ) -> None:
      
        self.meta = pd.read_csv(data_dir)
        self.all_images = list(self.meta.path)
        self.transform = transforms
        self.same = same
        
    def __len__(self
                ) -> int:
        return len(self.all_images)
    
    def __getitem__(self,
                    index: int
                    ) -> Tuple[torch.Tensor]:

        
        target_path = self.all_images[index]
        image_id = target_path.split('/')[-1][:-4]
        img_age_path, img_gender_path = self._retrieve_anchors(image_id=image_id,
                                                               meta = self.meta,
                                                               same=self.same)

        img_target = Image.open(fp=target_path).convert('RGB')
        img_target = self.transform(img_target)
        
        img_age = Image.open(fp=img_age_path).convert('RGB')
        img_age = self.transform(img_age)

        img_gender = Image.open(fp=img_gender_path).convert('RGB')
        img_gender = self.transform(img_gender)

        return (img_target,img_age,img_gender)
    
    
    def _retrieve_anchors(self,
                          image_id: str,
                          meta: pd.DataFrame,
                          same: bool = False) -> Tuple[str]:
        record = meta[meta.dicom_id == image_id]
    
        subject_id = list(record.subject_id)[0]
        age_groub =list(record.ageR5)[0] 
        gender = list(record.gender)[0]
    
        group = meta[meta.ageR5 == age_groub]
    
        if same:
            candidate_anchors = group[group.gender == gender]
            candidate_anchors = candidate_anchors[candidate_anchors.subject_id != subject_id]
            images= list(candidate_anchors.path)
            sampled_images = random.sample(images,k=2)
            image_age, image_gender = sampled_images[0],sampled_images[1]
            return image_age, image_gender
        else:
            candidate_anchors = group
            candidate_anchors = candidate_anchors[candidate_anchors.subject_id != subject_id]
            images= list(candidate_anchors.path)
            image_age = random.sample(images,k=1)[0]
            candidate_anchors = candidate_anchors[candidate_anchors.gender == gender]
            images= list(candidate_anchors.path)
            image_gender = random.sample(images,k=1)[0]
            return image_age, image_gender

In [38]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data in tqdm(dataloader):
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [39]:
transforms = MSNTransform()
dataset = ChexMSNDataset(data_dir='../data/meta.csv',transforms= transforms,same=True)
dataloader = DataLoader(dataset=dataset,batch_size=10,num_workers=24,pin_memory=True,shuffle=True)

In [50]:
import torch.nn.functional as F
class ChexMSNModel(pl.LightningModule):
    def __init__(self,
                 model: nn.Module,
                 criterion: nn.Module,
                 num_prototypes: int = 1024,
                 learning_rate: float =  1e-3,
                 weight_decay: float = 0.0,
                 max_epochs: int = 50,
                 focal: bool = True

                ) -> None:
        super().__init__()

        self.model = model
        self.criterion = criterion
        self.num_prototypes = num_prototypes
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.max_epochs = max_epochs
        self.focal = focal
        self.in_prototypes = self.model.projection_head.out_features
        self.prototypes = nn.ModuleList([nn.Linear(in_features=self.in_prototypes,out_features=num_prototypes,bias=False),
                                         nn.Linear(in_features=self.in_prototypes,out_features=num_prototypes,bias=False),
                                         nn.Linear(in_features=self.in_prototypes,out_features=num_prototypes,bias=False)])
    
    
    def training_step(self, 
                      batch: List[torch.Tensor], 
                      batch_idx: int
                     ) -> float:
        
        
        anchors, target = self.model(batch) 
        print('\n',target)
        loss = self.criterion(anchors,target,self.prototypes,focal=self.focal)
        self.log("train_loss", loss, on_epoch= True,on_step=True, logger=True,)
        print('\n',loss.detach())
        return loss


    def configure_optimizers(self):

        params = [
            *list(self.model.backbone.parameters()),
            *list(self.model.projection_head.parameters()),
            self.prototypes[0].weight,
            self.prototypes[1].weight,
            self.prototypes[2].weight,
        ]
        
        optimizer = torch.optim.AdamW(params, 
                                      lr=self.learning_rate,
                                      weight_decay=self.weight_decay)
        
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                               eta_min=0.00001,
                                                               T_max=self.max_epochs)
        return {'optimizer': optimizer,
               'lr_scheduler': scheduler
               }


In [51]:

transforms = MSNTransform()
dataset = ChexMSNDataset(data_dir='../data/meta.csv',transforms= transforms,same=True)
dataloader = DataLoader(dataset=dataset,batch_size=64,num_workers=24,pin_memory=True,shuffle=True)

In [52]:
criterterion = MSNLoss(temperature=0.1,
                       sinkhorn_iterations=0,
                       similarity_weight=1.0,
                       age_weight=0.0,
                       gender_weight=0.0,
                       regularization_weight=5)

In [53]:
model = ChexMSNModel(msn,criterterion,focal=True)



In [54]:
trainer = pl.Trainer(devices="auto",
                     accelerator="auto",
                     strategy="auto",
                     log_every_n_steps=1,
                     max_epochs=50,
                     precision=16,
                     )

/home/sas10092/.conda/envs/chexmsn-env/lib/python3.9/site-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/home/sas10092/.conda/envs/chexmsn-env/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/sas10092/.conda/envs/chexmsn-env/lib/python3.9 ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | model      | ChexMSN    | 13.2 M
1 | criterion  | MSNLoss    | 0     
2 | prototypes | ModuleList | 589 K 
------------------------------------------
7.2 M     Trainable params
6.6 M     Non-trainable params
13.8 M    Total params
55.207    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/5879 [00:00<?, ?it/s] 
 tensor([[[-0.9858, -2.0000,  0.5649,  ...,  1.3418, -0.1093, -1.0986],
         [-0.9985, -2.0723,  0.5146,  ...,  1.2842, -0.0777, -1.0557],
         [-0.9292, -1.7441,  0.6270,  ...,  1.4316, -0.2272, -1.1270]],

        [[-0.9849, -1.9844,  0.5679,  ...,  1.3447, -0.1246, -1.1055],
         [-0.9980, -2.0723,  0.5132,  ...,  1.2783, -0.0837, -1.0586],
         [-0.9097, -1.6914,  0.6328,  ...,  1.4375, -0.2483, -1.1143]],

        [[-0.4844,  0.3589,  0.0652,  ...,  0.1214,  2.1953, -1.8428],
         [-0.4153,  0.3186,  0.0768,  ...,  0.1132,  2.0527, -1.7744],
         [-0.4797,  0.5215, -0.0115,  ...,  0.1394,  2.5078, -1.8115]],

        ...,

        [[-1.0059, -2.0977,  0.5195,  ...,  1.2715, -0.0591, -1.0654],
         [-1.0137, -2.1738,  0.4424,  ...,  1.1816, -0.0419, -0.9946],
         [-0.9019, -1.6621,  0.6338,  ...,  1.4434, -0.2625, -1.1143]],

        [[-1.0107, -2.1426,  0.4912,  ...,  1.2275, -0.0394, -1.0352],
   

Epoch 0:   0%|          | 6/5879 [00:16<4:25:09,  0.37it/s, v_num=7]
 tensor([[[-1.0244, -2.2578,  0.4160,  ...,  1.1162,  0.0383, -0.9971],
         [-1.0127, -2.2988,  0.2881,  ...,  0.9722,  0.0102, -0.8833],
         [-0.8657, -1.5977,  0.6245,  ...,  1.4453, -0.2639, -1.1260]],

        [[-0.1627,  0.2820, -0.6641,  ...,  0.1545,  0.1186, -0.8262],
         [-0.1456, -0.0912, -0.8613,  ..., -0.0031, -0.4609, -0.3669],
         [-0.4397,  0.6753, -0.0984,  ...,  0.1534,  2.3359, -1.6709]],

        [[-1.0186, -2.1621,  0.5015,  ...,  1.2373,  0.0156, -1.0830],
         [-1.0264, -2.2285,  0.4214,  ...,  1.1436,  0.0245, -1.0088],
         [-0.9092, -1.7129,  0.6260,  ...,  1.4316, -0.2045, -1.1514]],

        ...,

        [[-0.3811,  0.3171, -0.1218,  ...,  0.1633,  1.4473, -1.5625],
         [-0.2141,  0.2260, -0.2959,  ...,  0.1438,  0.8057, -1.1719],
         [-0.4863,  0.5337,  0.0215,  ...,  0.1779,  2.4648, -1.7812]],

        [[-0.3108, -0.6084, -0.9683,  ..., -0.0640, -0.7

Epoch 0:   0%|          | 12/5879 [00:22<3:03:11,  0.53it/s, v_num=7]
 tensor([[[-3.0151e-01, -6.5674e-01, -1.0176e+00,  ..., -1.2469e-01,
          -7.2900e-01,  3.0762e-02],
         [-3.3179e-01, -9.2432e-01, -9.9512e-01,  ..., -2.5269e-01,
          -6.7480e-01,  8.3008e-02],
         [-3.8818e-01,  8.5205e-01, -1.9519e-01,  ...,  1.9470e-01,
           2.0488e+00, -1.4736e+00]],

        [[-9.1455e-01, -2.2910e+00,  8.9502e-04,  ...,  5.8057e-01,
           7.0429e-04, -5.8887e-01],
         [-8.7451e-01, -2.2402e+00, -1.2976e-01,  ...,  4.3604e-01,
          -6.1676e-02, -4.9683e-01],
         [-7.7002e-01, -1.4023e+00,  5.6982e-01,  ...,  1.4355e+00,
          -3.1494e-01, -1.0967e+00]],

        [[-1.0127e+00, -2.1621e+00,  5.0537e-01,  ...,  1.2520e+00,
           8.0566e-02, -1.1631e+00],
         [-1.0264e+00, -2.2402e+00,  4.4092e-01,  ...,  1.1709e+00,
           1.0828e-01, -1.1045e+00],
         [-9.1797e-01, -1.8008e+00,  5.9912e-01,  ...,  1.3994e+00,
          -9.8511

Epoch 0:   0%|          | 17/5879 [00:25<2:24:49,  0.67it/s, v_num=7]
 tensor([[[-0.3765, -0.9741, -0.9839,  ..., -0.2211, -0.6709,  0.1041],
         [-0.3906, -1.1367, -0.9727,  ..., -0.3179, -0.6128,  0.1066],
         [-0.2595,  1.0830, -0.3743,  ...,  0.3679,  1.0732, -1.1641]],

        [[-0.8862, -2.2793, -0.1307,  ...,  0.3987,  0.0242, -0.5308],
         [-0.7700, -2.0996, -0.3774,  ...,  0.1229, -0.1216, -0.3250],
         [-0.7783, -1.4434,  0.5444,  ...,  1.4170, -0.2455, -1.1367]],

        [[-0.5518,  0.1718,  0.2910,  ...,  0.3020,  1.9863, -1.7686],
         [-0.5127,  0.1536,  0.2979,  ...,  0.2883,  1.9121, -1.7354],
         [-0.5942,  0.3213,  0.2822,  ...,  0.3184,  2.4570, -1.8301]],

        ...,

        [[-0.9463, -2.3594, -0.0192,  ...,  0.5415,  0.0911, -0.6655],
         [-0.8491, -2.2168, -0.2524,  ...,  0.2732, -0.0454, -0.4622],
         [-0.8325, -1.5801,  0.5625,  ...,  1.4180, -0.1744, -1.1758]],

        [[-0.7856, -2.1172, -0.3079,  ...,  0.2023, -0.

...

Epoch 0:  25%|██▌       | 1491/5879 [16:15<47:49,  1.53it/s, v_num=7]
 tensor([[[-4.2461,  0.8618,  3.0684,  ..., -1.6924, -1.4287,  1.4170],
         [-4.2578,  0.8711,  3.0840,  ..., -1.6992, -1.4336,  1.4053],
         [-0.9268, -2.3906, -5.3789,  ...,  5.7305, -4.9219, -0.5454]],

        [[-6.7344,  2.7637,  6.7930,  ..., -3.8848, -2.3574, -2.7793],
         [-6.7344,  2.7637,  6.7930,  ..., -3.8848, -2.3594, -2.7812],
         [-1.0254, -2.2969, -5.4648,  ...,  6.0273, -4.9531, -0.4653]],

        [[-5.5195,  1.8623,  4.8945,  ..., -2.6641, -2.0000, -0.2107],
         [-5.6055,  1.9238,  5.0156,  ..., -2.7324, -2.0352, -0.3296],
         [-0.9800, -2.3359, -5.4336,  ...,  5.8945, -4.9531, -0.5010]],

        ...,

        [[-4.9102,  1.4062,  4.0273,  ..., -2.1777, -1.7393,  0.6079],
         [-4.8555,  1.3652,  3.9512,  ..., -2.1387, -1.7158,  0.6758],
         [-0.9385, -2.3770, -5.3906,  ...,  5.7656, -4.9297, -0.5352]],

        [[-6.5625,  2.6777,  6.6680,  ..., -3.8945, -2.

Epoch 0:  25%|██▌       | 1497/5879 [16:17<47:40,  1.53it/s, v_num=7]
 tensor([[[-2.1816, -1.5596,  0.0429,  ..., -0.6914, -0.1299,  3.2109],
         [-2.1855, -1.5557,  0.0464,  ..., -0.6919, -0.1316,  3.2090],
         [-0.8013, -2.5312, -5.2148,  ...,  5.3438, -4.8203, -0.6235]],

        [[-6.5273,  2.6016,  6.4258,  ..., -3.5879, -2.3828, -1.9746],
         [-6.5273,  2.6016,  6.4258,  ..., -3.5879, -2.3828, -1.9775],
         [-1.0166, -2.3066, -5.4414,  ...,  6.0117, -4.9727, -0.4446]],

        [[-5.0156,  1.4834,  4.1992,  ..., -2.2383, -1.7842,  0.4492],
         [-5.0195,  1.4844,  4.1992,  ..., -2.2402, -1.7842,  0.4473],
         [-0.9873, -2.3340, -5.4180,  ...,  5.9219, -4.9766, -0.4695]],

        ...,

        [[-5.4922,  1.8447,  4.8906,  ..., -2.6270, -1.9941, -0.2035],
         [-5.5664,  1.9004,  5.0000,  ..., -2.6895, -2.0254, -0.3118],
         [-0.9707, -2.3496, -5.4023,  ...,  5.8711, -4.9688, -0.4844]],

        [[-3.9043,  1.3115,  3.1680,  ..., -2.2637, -1.

Epoch 0:  26%|██▌       | 1503/5879 [16:23<47:43,  1.53it/s, v_num=7]
 tensor([[[-6.6172,  2.7188,  6.7578,  ..., -3.9199, -2.3164, -3.1367],
         [-6.6172,  2.7188,  6.7578,  ..., -3.9199, -2.3164, -3.1367],
         [-1.0312, -2.2988, -5.4219,  ...,  6.0469, -4.9766, -0.4094]],

        [[-6.6523,  2.7168,  6.7148,  ..., -3.8105, -2.4043, -2.6055],
         [-6.6523,  2.7148,  6.7070,  ..., -3.8027, -2.4062, -2.5801],
         [-1.0156, -2.3125, -5.4219,  ...,  6.0117, -4.9961, -0.4192]],

        [[-6.2188,  2.3789,  5.9883,  ..., -3.2773, -2.2949, -1.4150],
         [-6.2109,  2.3730,  5.9766,  ..., -3.2715, -2.2930, -1.4023],
         [-1.0156, -2.3125, -5.4219,  ...,  6.0117, -4.9961, -0.4194]],

        ...,

        [[-4.6484,  1.1787,  3.6836,  ..., -1.9160, -1.6045,  0.9307],
         [-4.6484,  1.1797,  3.6855,  ..., -1.9160, -1.6045,  0.9292],
         [-0.9561, -2.3672, -5.3672,  ...,  5.8242, -4.9766, -0.4729]],

        [[-6.2539,  2.4043,  6.0391,  ..., -3.3145, -2.

Epoch 0:  26%|██▌       | 1509/5879 [16:24<47:32,  1.53it/s, v_num=7]
 tensor([[[-6.5547e+00,  2.6426e+00,  6.5703e+00,  ..., -3.6816e+00,
          -2.4141e+00, -2.3047e+00],
         [-6.5547e+00,  2.6426e+00,  6.5703e+00,  ..., -3.6836e+00,
          -2.4141e+00, -2.3105e+00],
         [-9.9805e-01, -2.3301e+00, -5.3906e+00,  ...,  5.9648e+00,
          -5.0156e+00, -4.0845e-01]],

        [[-4.0508e+00,  6.4844e-01,  2.8066e+00,  ..., -1.4316e+00,
          -1.2861e+00,  1.6670e+00],
         [-4.0508e+00,  6.4990e-01,  2.8105e+00,  ..., -1.4326e+00,
          -1.2871e+00,  1.6650e+00],
         [-9.6240e-01, -2.3633e+00, -5.3594e+00,  ...,  5.8555e+00,
          -5.0039e+00, -4.4141e-01]],

        [[-8.7793e-01, -3.8457e+00, -9.7607e-01,  ..., -1.1963e+00,
           8.7793e-01,  2.7285e+00],
         [-8.8086e-01, -3.8379e+00, -9.7559e-01,  ..., -1.1914e+00,
           8.7598e-01,  2.7324e+00],
         [ 4.3488e-03, -3.9551e+00, -3.3320e+00,  ...,  1.7607e+00,
          -2.9805

Epoch 0:  26%|██▌       | 1515/5879 [16:26<47:22,  1.54it/s, v_num=7]
 tensor([[[-5.1172,  1.5420,  4.4102,  ..., -2.2461, -1.8291,  0.2712],
         [-5.0039,  1.4531,  4.2461,  ..., -2.1504, -1.7744,  0.4304],
         [-1.0137, -2.3184, -5.3750,  ...,  6.0078, -5.0273, -0.3701]],

        [[-5.9453,  2.1758,  5.6367,  ..., -3.0137, -2.2129, -1.0264],
         [-5.9492,  2.1777,  5.6406,  ..., -3.0156, -2.2148, -1.0303],
         [-1.0186, -2.3164, -5.3828,  ...,  6.0312, -5.0312, -0.3640]],

        [[-5.9180,  2.3613,  6.0352,  ..., -3.7012, -1.9521, -3.7988],
         [-5.9180,  2.3613,  6.0352,  ..., -3.7012, -1.9521, -3.8008],
         [-1.0312, -2.3066, -5.3867,  ...,  6.0586, -5.0156, -0.3564]],

        ...,

        [[-1.7852, -2.2578, -0.4387,  ..., -0.5542,  0.3142,  3.3262],
         [-1.7998, -2.2344, -0.4241,  ..., -0.5508,  0.3030,  3.3262],
         [-0.7036, -2.6543, -5.0234,  ...,  5.0234, -4.7656, -0.6548]],

        [[-1.7100, -2.3750, -0.5083,  ..., -0.5679,  0.

In [46]:
!nvidia-smi

Tue Nov 21 13:54:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM-80GB       Off  | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    62W / 500W |      0MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-SXM-80GB       Off  | 00000000:41:00.0 Off |                    0 |
| N/A   

In [48]:
dist.is_available()

True